# Setup dependencies
I will be using pandas and sklearn for managing data and machine learning.
<details>
    <summary>pip install...</summary>

```python
# Allows to install a python package
pip install package-name
# or install python package with a specific version
pip install package-name==version
```
</details>


In [1]:
# Install PySpark version 3.1.2 silently
#!pip install pyspark==3.1.2 -q
# Install findSpark silently
!pip install findspark -q

In [2]:
# Used to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# Create Spark Session

In [2]:
import findspark

# Initializing FindSpark to locate Spark installation
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from datetime import datetime

spark = SparkSession.builder.appName("Feature Extraction and Transformation using Spark").getOrCreate()

# Creating Dataset

In [3]:
sentenceDataFrame = spark.createDataFrame(
    [(1, "Spark is a distributed computing system."),
    (2, "It provides interfaces for multiple languages"),
    (3, "Spark is built on top of Hadoop")], 
    
    ["id", "sentence"]
)
sentenceDataFrame.show(truncate = False)

+---+---------------------------------------------+
|id |sentence                                     |
+---+---------------------------------------------+
|1  |Spark is a distributed computing system.     |
|2  |It provides interfaces for multiple languages|
|3  |Spark is built on top of Hadoop              |
+---+---------------------------------------------+



# Tokenizer
It is used to break a sentence into words.

In [4]:
from pyspark.ml.feature import Tokenizer

# Instantiate
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
# Tokenize
token_df = tokenizer.transform(sentenceDataFrame)
# Display the tokenized data
token_df.show(truncate=False)

+---+---------------------------------------------+----------------------------------------------------+
|id |sentence                                     |words                                               |
+---+---------------------------------------------+----------------------------------------------------+
|1  |Spark is a distributed computing system.     |[spark, is, a, distributed, computing, system.]     |
|2  |It provides interfaces for multiple languages|[it, provides, interfaces, for, multiple, languages]|
|3  |Spark is built on top of Hadoop              |[spark, is, built, on, top, of, hadoop]             |
+---+---------------------------------------------+----------------------------------------------------+



# CountVectorizer
It is used to convert text into numerical format. It gives the count of each word in a given document.

It returns:
- Total number of words
- List of index of words (unordered) of the row
- Frequency of each word corresponding to these indexes.

In [5]:
#create a sample dataframe and display it.
textdata = [(1, "I love Spark Spark provides Python API ".split()),
            (2, "I love Python Spark supports Python".split()),
            (3, "Spark solves the big problem of big data".split())]

textdata = spark.createDataFrame(textdata, ["id", "words"])
textdata.show(truncate=False)

+---+-------------------------------------------------+
|id |words                                            |
+---+-------------------------------------------------+
|1  |[I, love, Spark, Spark, provides, Python, API]   |
|2  |[I, love, Python, Spark, supports, Python]       |
|3  |[Spark, solves, the, big, problem, of, big, data]|
+---+-------------------------------------------------+



In [9]:
from pyspark.ml.feature import CountVectorizer

# Create a CountVectorizer object
# Specify the column to be count vectorized as inputcol
# and the output column name where the count vectors are to be stored.
cv = CountVectorizer(inputCol="words", outputCol="features")

# Fit the CountVectorizer model on the input data
model = cv.fit(textdata)

# Transform the input data to bag-of-words vectors
result = model.transform(textdata)

# Print index and related word
print({ index: word for index, word in enumerate(model.vocabulary)})

# display the dataframe
result.show(truncate=False)

{0: 'Spark', 1: 'Python', 2: 'I', 3: 'love', 4: 'big', 5: 'the', 6: 'provides', 7: 'of', 8: 'API', 9: 'solves', 10: 'data', 11: 'problem', 12: 'supports'}
+---+-------------------------------------------------+----------------------------------------------------+
|id |words                                            |features                                            |
+---+-------------------------------------------------+----------------------------------------------------+
|1  |[I, love, Spark, Spark, provides, Python, API]   |(13,[0,1,2,3,6,8],[2.0,1.0,1.0,1.0,1.0,1.0])        |
|2  |[I, love, Python, Spark, supports, Python]       |(13,[0,1,2,3,12],[1.0,2.0,1.0,1.0,1.0])             |
|3  |[Spark, solves, the, big, problem, of, big, data]|(13,[0,4,5,7,9,10,11],[1.0,2.0,1.0,1.0,1.0,1.0,1.0])|
+---+-------------------------------------------------+----------------------------------------------------+



# TF-IDF
Term Frequency-Inverse Document Frequency is used to quantify the importance of a word in a document. TF-IDF is computed by multiplying the number of times a word occurs in a document by the inverse document frequency of the word.


In [10]:
#create a sample dataframe and display it.
sentenceData = spark.createDataFrame([
        (1, "Spark supports python"),
        (2, "Spark is fast"),
        (3, "Spark is easy")
    ], ["id", "sentence"])

sentenceData.show(truncate = False)

+---+---------------------+
|id |sentence             |
+---+---------------------+
|1  |Spark supports python|
|2  |Spark is fast        |
|3  |Spark is easy        |
+---+---------------------+



In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

#tokenize the "sentence" column and store in the column "words"
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show(truncate = False)

+---+---------------------+-------------------------+
|id |sentence             |words                    |
+---+---------------------+-------------------------+
|1  |Spark supports python|[spark, supports, python]|
|2  |Spark is fast        |[spark, is, fast]        |
|3  |Spark is easy        |[spark, is, easy]        |
+---+---------------------+-------------------------+



In [12]:
# Create a HashingTF object
# mention the "words" column as input
# mention the "rawFeatures" column as output

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10)
featurizedData = hashingTF.transform(wordsData)

featurizedData.show(truncate = False)

+---+---------------------+-------------------------+--------------------------+
|id |sentence             |words                    |rawFeatures               |
+---+---------------------+-------------------------+--------------------------+
|1  |Spark supports python|[spark, supports, python]|(10,[4,6,9],[1.0,1.0,1.0])|
|2  |Spark is fast        |[spark, is, fast]        |(10,[3,6,9],[1.0,1.0,1.0])|
|3  |Spark is easy        |[spark, is, easy]        |(10,[0,6,9],[1.0,1.0,1.0])|
+---+---------------------+-------------------------+--------------------------+



In [13]:
# Create an IDF object
# mention the "rawFeatures" column as input
# mention the "features" column as output

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tfidfData = idfModel.transform(featurizedData)

In [14]:
#display the tf-idf data
tfidfData.select("sentence", "features").show(truncate=False)

+---------------------+-----------------------------------------+
|sentence             |features                                 |
+---------------------+-----------------------------------------+
|Spark supports python|(10,[4,6,9],[0.6931471805599453,0.0,0.0])|
|Spark is fast        |(10,[3,6,9],[0.6931471805599453,0.0,0.0])|
|Spark is easy        |(10,[0,6,9],[0.6931471805599453,0.0,0.0])|
+---------------------+-----------------------------------------+



# StopWordsRemover
StopWordsRemover is a transformer that filters out stop words like "a","an" and "the".

In [17]:
from pyspark.ml.feature import StopWordsRemover
#create a dataframe with sample text and display it
textData = spark.createDataFrame([
    (1, ['Spark', 'is', 'an', 'open-source', 'distributed', 'computing', 'system']),
    (2, ['IT', 'has', 'interfaces', 'for', 'multiple', 'languages']),
    (3, ['It', 'has', 'a', 'wide', 'range', 'of', 'libraries', 'and', 'APIs'])
], ["id", "sentence"])

textData.show(truncate = False)

+---+------------------------------------------------------------+
|id |sentence                                                    |
+---+------------------------------------------------------------+
|1  |[Spark, is, an, open-source, distributed, computing, system]|
|2  |[IT, has, interfaces, for, multiple, languages]             |
|3  |[It, has, a, wide, range, of, libraries, and, APIs]         |
+---+------------------------------------------------------------+



In [18]:
# remove stopwords from "sentence" column and store the result in "filtered_sentence" column
remover = StopWordsRemover(inputCol="sentence", outputCol="filtered_sentence")
textData = remover.transform(textData)

textData.show(truncate = False)

+---+------------------------------------------------------------+----------------------------------------------------+
|id |sentence                                                    |filtered_sentence                                   |
+---+------------------------------------------------------------+----------------------------------------------------+
|1  |[Spark, is, an, open-source, distributed, computing, system]|[Spark, open-source, distributed, computing, system]|
|2  |[IT, has, interfaces, for, multiple, languages]             |[interfaces, multiple, languages]                   |
|3  |[It, has, a, wide, range, of, libraries, and, APIs]         |[wide, range, libraries, APIs]                      |
+---+------------------------------------------------------------+----------------------------------------------------+



# StringIndexer
StringIndexer converts a column of strings into a column of integers.

In [20]:
from pyspark.ml.feature import StringIndexer

colors = spark.createDataFrame(
    [(0, "red"), (1, "red"), (2, "blue"), (3, "yellow" ), (4, "yellow"), (5, "yellow")],
    ["id", "color"])

colors.show()

+---+------+
| id| color|
+---+------+
|  0|   red|
|  1|   red|
|  2|  blue|
|  3|yellow|
|  4|yellow|
|  5|yellow|
+---+------+



In [21]:
# index the strings in the column "color" and store their indexes in the column "colorIndex"
indexer = StringIndexer(inputCol="color", outputCol="colorIndex")
indexed = indexer.fit(colors).transform(colors)

# display the dataframe
indexed.show()

+---+------+----------+
| id| color|colorIndex|
+---+------+----------+
|  0|   red|       1.0|
|  1|   red|       1.0|
|  2|  blue|       2.0|
|  3|yellow|       0.0|
|  4|yellow|       0.0|
|  5|yellow|       0.0|
+---+------+----------+



# StandardScaler
StandardScaler transforms the data so that it has a mean of 0 and a standard deviation of 1

In [22]:
from pyspark.ml.feature import StandardScaler

# Create a sample dataframe and display it
from pyspark.ml.linalg import Vectors
data = [(1, Vectors.dense([70, 170, 17])),
        (2, Vectors.dense([80, 165, 25])),
        (3, Vectors.dense([65, 150, 135]))]
df = spark.createDataFrame(data, ["id", "features"])

df.show()

+---+------------------+
| id|          features|
+---+------------------+
|  1| [70.0,170.0,17.0]|
|  2| [80.0,165.0,25.0]|
|  3|[65.0,150.0,135.0]|
+---+------------------+



In [23]:
# Define the StandardScaler transformer
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

# Fit the transformer to the dataset
scalerModel = scaler.fit(df)

# Scale the data
scaledData = scalerModel.transform(df)

# Show the scaled data
scaledData.show(truncate = False)

+---+------------------+-----------------------------------------------------------+
|id |features          |scaledFeatures                                             |
+---+------------------+-----------------------------------------------------------+
|1  |[70.0,170.0,17.0] |[-0.218217890235993,0.8006407690254367,-0.6369487984517485]|
|2  |[80.0,165.0,25.0] |[1.0910894511799611,0.3202563076101752,-0.5156252177942725]|
|3  |[65.0,150.0,135.0]|[-0.8728715609439701,-1.120897076635609,1.152574016246021] |
+---+------------------+-----------------------------------------------------------+



# Stop SparkSession

In [16]:
spark.stop()